In [ ]:
!pip install statsmodels
!pip install hyperas
!pip install hyperopt

In [ ]:
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from tensorflow.keras import regularizers
from keras import backend as K

from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform

from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import Session
import gc

In [ ]:
def data():
    ##Import data from hd5f file
    PATH_3D_H5 = 'Images/preprocessed_data_128_112_40.hdf5'
    with h5py.File(PATH_3D_H5, 'r') as h5:
        print('H5-file: ', list(h5.keys()))

         # Training data
        X_train = h5["X_train"][:]
        Y_train = h5["Y_train"][:]
        pat_train = h5["pat_train"][:]
        
        # Validation data
        X_valid = h5["X_valid"][:]
        Y_valid = h5["Y_valid"][:]
        pat_valid = h5["pat_valid"][:]

        # Test
        X_test = h5["X_test"][:]
        Y_test = h5["Y_test"][:]
        pat_test = h5["pat_test"][:]

    print(X_train.shape, X_train.min(), X_train.max(), X_train.mean(), X_train.std(), Y_train.shape)
    print(X_valid.shape, X_valid.min(), X_valid.max(), X_valid.mean(), X_valid.std(), Y_valid.shape)
    print(X_test.shape, X_test.min(), X_test.max(), X_test.mean(), X_test.std(), Y_test.shape)
    
    ###Create balanced dataset
    bool_train_labels = Y_train[:,1] != 0
    pos_features = X_train[bool_train_labels]
    neg_features = X_train[~bool_train_labels]
    pos_labels = Y_train[bool_train_labels]
    neg_labels = Y_train[~bool_train_labels]

    ids = np.arange(len(neg_features))
    choices = np.random.choice(ids, len(pos_features))
    res_neg_features = neg_features[choices]
    res_neg_labels = neg_labels[choices]

    resampled_features = np.concatenate([res_neg_features, pos_features], axis=0)
    resampled_labels = np.concatenate([res_neg_labels, pos_labels], axis=0)

    order = np.arange(len(resampled_labels))
    np.random.shuffle(order)
    X_train_balanced = resampled_features[order]
    Y_train_balanced = resampled_labels[order]

    print(X_train_balanced.shape, Y_train_balanced.shape)
    
    return X_train_balanced, Y_train_balanced, X_valid, Y_valid

In [ ]:
def model(X_train, Y_train, X_valid, Y_valid):  
    num_classes = 2
    model = keras.Sequential()
    model.add(layers.Convolution3D({{choice([8, 16, 32])}}, kernel_size=3, activation="relu", 
                                   batch_input_shape=(None,128,112,40,1), padding = 'same'))
    model.add(layers.MaxPooling3D(pool_size = 2))
    model.add(layers.BatchNormalization(center = True, scale = True))
    
    model.add(layers.Convolution3D({{choice([8, 16, 32])}}, kernel_size=3, activation="relu", padding = 'same'))
    model.add(layers.MaxPooling3D(pool_size = 2))
    model.add(layers.BatchNormalization(center = True, scale = True))
    
    model.add(layers.Convolution3D({{choice([16, 32, 64])}}, kernel_size=3, activation="relu", padding = 'same'))
    model.add(layers.MaxPooling3D(pool_size = 2))
    model.add(layers.BatchNormalization(center = True, scale = True))
    
    model.add(layers.Convolution3D({{choice([16, 32, 64])}}, kernel_size=3, activation="relu", padding = 'same'))
    model.add(layers.MaxPooling3D(pool_size = 2))
    model.add(layers.BatchNormalization(center = True, scale = True))
    
    model.add(layers.Convolution3D({{choice([32, 64, 128])}}, kernel_size=3, activation="relu", padding = 'same'))
    model.add(layers.MaxPooling3D(pool_size = 2, padding = 'same'))
    model.add(layers.BatchNormalization(center = True, scale = True))
    
    model.add(layers.Flatten())
    model.add(layers.Dense({{choice([32, 64, 128])}}, activation="relu"))
    model.add(layers.Dropout({{uniform(0, 0.6)}}))
    
    model.add(layers.Dense(num_classes, activation='softmax'))
              
    #compile model
    metrics = [
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.AUC(name='auc')]

    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(learning_rate = 0.0001),
                  metrics=metrics)
    
    history = model.fit(X_train_balanced, Y_train_balanced,
                        validation_data=(X_valid, Y_valid), 
                        batch_size=2,
                        verbose=2,
                        epochs=50)
    
    #get the highest validation accuracy of the training epochs
    val_auc = np.amax(history.history['val_auc']) 
    print('Best validation auc of epoch:', val_auc)
              
    return {'loss': val_auc, 'status': STATUS_OK, 'model': model}

In [ ]:
best_run, best_model = optim.minimize(model=model,
                                      data=data,
                                      max_evals=25,
                                      algo=tpe.suggest,
                                      notebook_name="Modell_Schlaganfall_Detektion_Hyperparameter_Optimierung", # Without this it can't find the notebook!
                                      trials=Trials())

In [ ]:
best_run

In [ ]:
best_model.summary()